# TP1

# Problema 1
#### Pretende-se construir um horário semanal para o plano de reuniões de projeto de uma “StartUp” de acordo com as seguintes condições:
 a. Cada reunião ocupa uma sala (enumeradas $1...S,$) durante um “slot”  $1..T,$$ $$(\text{hora},\text{dia})$.
 <br>
 <br>
 b. Cada reunião tem associado um projeto (enumerados $1..P$) e um conjunto de participantes. Os diferentes colaboradores são enumerados $1..C$, em que $c = 1$ é o líder do projeto.
 <br>
 <br>
 c. Cada projeto tem associado um conjunto de colaboradores, dos quais um  é o líder. Cada projeto realiza um dado número de reuniões semanais. 
 <br>
 <br>
 d. O líder do projeto participa em todas as reuniões do seu projeto; os restantes colaboradores podem ou não participar consoante a sua disponibilidade, num mínimo (“quorum”) de  $50\%$ do total de colaboradores do projeto.

### Variáveis
Serão definidos 2 dicionários que representam as seguintes variáveis:
<br>
$ a_{c,\,p,\,d,\,h}$: representa a disponibilidade do colaborador $\textit{c}$ do projeto $\textit{p}$ no slot $\textit{(d,h)}$.
<br>
$ b_{p,\,s,\,d,\,h}$: representa a existência de uma reunião do projeto $\textit{p}$ na sala $\textit{s}$ no slot $\textit{(d,h)}$.

Todas as variáveis são binárias, ou seja, apenas podem tomar como valores `0` ou `1`.
<br>
<br>
$
0 \leq a_{c,\,p,\,d,\,h} \leq 1, \forall_{c \leq C,\,p \leq P,\,d \leq D,\, h \leq H}
$
<br>
<br>
$
0 \leq b_{s,\,p,\,d,\,h} \leq 1, \forall_{s \leq S,\, p \leq P,\, d \leq D,\,h \leq H}
$

### Implementação
Começamos por importar a biblioteca de programação linear do OR-Tools e criar uma instância do solver.
<br>
Entretanto, inicializamos o *solver* `horario` e declaramos as matrizes de alocação, $\textit{a}$ e $\textit{b}$.

In [19]:
from ortools.linear_solver import pywraplp

horario = pywraplp.Solver.CreateSolver('SCIP')

S = 2
T = 30
P = 3
C = 8
D = 5
H = 5

a = {}
for c in range(1, C+1):
    a[c] = {}
    for p in range(1, P+1):
        a[c][p] = {}
        for d in range(1, D+1):
            a[c][p][d] = {}
            for h in range(1, H+1):
                a[c][p][d][h] = horario.BoolVar(f'a[{c}][{p}][{d}][{h}]')
            
            
b = {}
for p in range(1, P+1):
    b[p] = {}
    for s in range(1, S+1):
        b[p][s] = {}
        for d in range(1, D+1):
            b[p][s][d] = {}
            for h in range(1, H+1):
                b[p][s][d][h] = horario.BoolVar(f'b[{p}][{s}][{d}][{h}]')
            

In [20]:
projetos = {
    1: (1, 8, [1,3,5,8]),
    2: (2, 7, [1,2,4,7]),
    3: (3, 9, [2,3,5,6,7,8])
}

colaboradores = {
    1: [(1,1),(1,2),(1,3),(1,4),(1,5),(2,1),(2,2),(2,3),(2,4),(2,5),(3,1),(3,2),(3,3),(3,4),(3,5),
       (4,1),(4,2),(4,3),(4,4),(4,5),(5,1),(5,2),(5,3),(5,4),(5,5)],
    2: [(1,1),(1,2),(1,3),(1,4),(1,5),(2,1),(2,2),(2,3),(2,4),(2,5),(3,1),(3,2),(3,3),(3,4),(3,5),
       (4,1),(4,2),(4,3),(4,4),(4,5),(5,1),(5,2),(5,3),(5,4),(5,5)],
    3: [(1,1),(1,2),(1,3),(1,4),(1,5),(2,1),(2,2),(2,3),(2,4),(2,5),(3,1),(3,2),(3,3),(3,4),(3,5),
       (4,1),(4,2),(4,3),(4,4),(4,5),(5,1),(5,2),(5,3),(5,4),(5,5)],
    4: [(1,1),(1,2),(1,3),(1,4),(1,5),(2,1),(2,2),(2,3),(2,4),(2,5),(3,1),(3,2),(3,3),(3,4),(3,5),
       (4,1),(4,2),(4,3),(4,4),(4,5),(5,1),(5,2),(5,3),(5,4),(5,5)],
    5: [(1,1),(1,2),(1,3),(1,4),(1,5),(2,1),(2,2),(2,3),(2,4),(2,5),(3,1),(3,2),(3,3),(3,4),(3,5),
       (4,1),(4,2),(4,3),(4,4),(4,5),(5,1),(5,2),(5,3),(5,4),(5,5)],
    6: [(1,1),(1,2),(1,3),(1,4),(1,5),(2,1),(2,2),(2,3),(2,4),(2,5),(3,1),(3,2),(3,3),(3,4),(3,5),
       (4,1),(4,2),(4,3),(4,4),(4,5),(5,1),(5,2),(5,3),(5,4),(5,5)],
    7: [(1,1),(1,2),(1,3),(1,4),(1,5),(2,1),(2,2),(2,3),(2,4),(2,5),(3,1),(3,2),(3,3),(3,4),(3,5),
       (4,1),(4,2),(4,3),(4,4),(4,5),(5,1),(5,2),(5,3),(5,4),(5,5)],
    8: [(1,1),(1,2),(1,3),(1,4),(1,5),(2,1),(2,2),(2,3),(2,4),(2,5),(3,1),(3,2),(3,3),(3,4),(3,5),
       (4,1),(4,2),(4,3),(4,4),(4,5),(5,1),(5,2),(5,3),(5,4),(5,5)]
}

#### Restrição 1
Cada sala, num determinado slot, está disponível apenas para uma reunião de um projeto.
<br>
<br>
$ \forall_{s \leq S,\,d \leq D,\,h \leq H}. \sum_{p=1}^{P} b_{p,\,s,\,d,\,h} \leq 1 $

In [21]:
for s in range(1, S+1):
    for d in range(1, D+1):
        for h in range(1, H+1):
            horario.Add(sum([b[p][s][d][h] for p in range(1, P+1)]) <= 1)

#### Restrição 2
A reunião de um projeto realiza-se apenas numa das salas.

$ \forall_{p \leq P,\, d \leq D,\, h \leq H}. \sum_{s=1}^{S} b_{p,\,s,\,d,\,h} = 1 $

In [22]:
for p in range(1, P+1):
    for d in range(1, D+1):
        for h in range(1, H+1):
            horario.Add(sum([b[p][s][d][h] for s in range(1, S+1)]) <= 1)

#### Restrição 3
Pelo menos $50\%$ do total de colaboradores do projeto participam na reunião.
<br>
<br>
$ \forall_{p \leq P,\, d \leq D,\,h \leq H}. \sum_{i = 0}^{len(c)-1} a_{c[i],\,p,\,d,\,h} \geq \frac{len(c) - 1}{2} * a_{l,\,p,\,d,\,h},\, c = projetos_{p,\,2},\, l = projetos_{p,0}$

In [23]:
for p in range(1, P+1):
        for d in range(1, D+1):
            for h in range(1, H+1):
                c = projetos[p][2]
                l = projetos[p][0]
                horario.Add(sum([a[c[i]][p][d][h] for i in range(len(c))]) >= (len(c)/2) * a[l][p][d][h])

#### Restrição 4
O líder do projeto participa em todas as reuniões do seu projeto.

$ \forall_{p \leq P,\,d \leq D,\,h \leq H}. \sum_{s=1}^{S} b_{p,\,s,\,d,\,h} = a_{l,\,p,\,d,\,h},\, l = projetos_{p,0} $

In [24]:
for p in range(1, P+1):
        for d in range(1, D+1):
            for h in range(1, H+1):
                c = projetos[p][2]
                l = projetos[p][0]
                horario.Add(sum([b[p][s][d][h] for s in range(1, S+1)]) == a[l][p][d][h])

#### Restrição 5
Cada projeto realiza, no mínimo, um dado número de reuniões semanais.
<br>
<br>
$ \forall_{p \leq P}. \sum_{s=1}^{S} \sum_{d=1}^{D} \sum_{h=1}^{H} b_{p,\,s,\,d,\,h} \geq projetos_{p,\,1} $

In [25]:
for p in range(1, P+1):
    horario.Add(sum([b[p][s][d][h] for s in range(1, S+1) for d in range(1, D+1) for h in range(1, H+1)]) >= projetos[p][1])

#### Restrição 6
Cada colaborador tem apenas uma reunião num determinado slot.
<br>
<br>
$ \forall_{d \leq D,\,h \leq H,\, c \leq C} \sum_{p = 1}^{P} a_{c,\,p,\,d,\,h} \leq 1 $

In [26]:
for d in range(1, D+1):
    for h in range(1, H+1):
        for c in range(1, C+1):
            horario.Add(sum([a[c][p][d][h] for p in range(1, P+1)]) <= 1)

#### Restrição 7
Cada colaborador só pode ser alocado aos projetos em que está incluído.

$ \forall_{c \leq C,\,p \leq P,\,d \leq D,\, h \leq H}. c \notin projetos_{p,\,2} \implies a_{c,\,p,\,d,\,h} = 0 $

In [27]:
for c in range(1, C+1):
    for p in range(1, P+1):
        for d in range(1, D+1):
            for h in range(1, H+1):
                if c not in projetos[p][2]:
                    horario.Add(a[c][p][d][h] == 0)

#### Restrição 8
Cada colaborador só pode ser alocado aos slots em que está disponível.

$ \forall_{c \leq C,\,p \leq P,\,d \leq D,\, h \leq H}. (d,h) \notin colaboradores_{c} \implies a_{c,\,p,\,d,\,h} = 0 $

In [28]:
for c in range(1, C+1):
    for p in range(1, P+1):
        for d in range(1, D+1):
            for h in range(1, H+1):
                if (d,h) not in colaboradores[c]:
                    horario.Add(a[c][p][d][ħ] == 0)

### Maximizar o número de reúniões realizadas por projeto 

In [29]:
horario.Maximize(sum([b[p][s][d][h] for s in range(1, S+1) for p in range(1,P+1) for d in range(1, D+1) for h in range(1, H+1)]))

### Minimizar o número médio de reuniões semanais por colaborador 

In [30]:
horario.Minimize(sum([a[c][p][d][h] for c in range(1,C+1) for p in range(1,P+1) for d in range(1,D+1) for h in range(1,H+1)]))

### Solução do Problema 

In [34]:
status = horario.Solve()

if status == pywraplp.Solver.OPTIMAL:
    for d in range(1, D+1):
        print("---- Dia" + str(d) + "----\n")
        for h in range(1, H+1):
            for s in range(1, S+1):
                for p in range(1, P+1):
                    if int(b[p][s][d][h].solution_value()) == 1:
                        print("Hora: ", h)
                        print("Sala: ", s)
                        print("Projeto: ", p)
                        print("Líder: ", projetos[p][0])
                        temp = projetos[p][2]
                        for c in temp:
                            if int(a[c][p][d][h].solution_value()) == 1:
                                print("Colaborador: ", c)
else:
    print("Não existe solução")
                        

---- Dia1----

---- Dia2----

Hora:  5
Sala:  2
Projeto:  3
Líder:  3
Colaborador:  3
Colaborador:  6
Colaborador:  8
---- Dia3----

Hora:  1
Sala:  2
Projeto:  3
Líder:  3
Colaborador:  3
Colaborador:  6
Colaborador:  8
Hora:  2
Sala:  2
Projeto:  3
Líder:  3
Colaborador:  3
Colaborador:  6
Colaborador:  8
Hora:  3
Sala:  2
Projeto:  3
Líder:  3
Colaborador:  3
Colaborador:  6
Colaborador:  8
Hora:  4
Sala:  2
Projeto:  3
Líder:  3
Colaborador:  3
Colaborador:  6
Colaborador:  8
Hora:  5
Sala:  2
Projeto:  3
Líder:  3
Colaborador:  3
Colaborador:  6
Colaborador:  8
---- Dia4----

Hora:  1
Sala:  2
Projeto:  3
Líder:  3
Colaborador:  3
Colaborador:  6
Colaborador:  8
Hora:  2
Sala:  2
Projeto:  3
Líder:  3
Colaborador:  3
Colaborador:  6
Colaborador:  8
Hora:  3
Sala:  1
Projeto:  3
Líder:  3
Colaborador:  3
Colaborador:  6
Colaborador:  7
Hora:  3
Sala:  2
Projeto:  1
Líder:  1
Colaborador:  1
Colaborador:  8
Hora:  4
Sala:  1
Projeto:  2
Líder:  2
Colaborador:  2
Colaborador:  4
Hora